Spotify Recommendation Algorithm 
Steps:
1. Install required packages and dependencies and read csv file containing song data
2. Clean CSV data and convert dataframe into item-feature matrix 
3. Read spotify song playlist URL and gather playlist data from the Spotify API
4. Make sure playlist dataframe and song database dataframe have the same corresponding features (columns) 
5. Compute the cosine similarity between the playlist and database of songs to recommend songs 
6. Show recommendations to user! 

In [1]:
# Install packages and dependencies
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import spotipy
import json
from spotipy.oauth2 import SpotifyOAuth
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [19]:
# Read CSV file 
# @st.cache_data
feat_vec = pd.read_csv('data\cleaned_spotify_data.csv')
genre_list_df = pd.read_csv('data\genres_data.csv')

# Convert DataFrame to a list of rows
genre_list = genre_list_df['Genre'].tolist()

pd.set_option('display.max_columns', None)

genre_list

<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\g'
<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Quan\AppData\Local\Temp\ipykernel_12452\4019313650.py:3: SyntaxWarning: invalid escape sequence '\c'
  feat_vec = pd.read_csv('data\cleaned_spotify_data.csv')
C:\Users\Quan\AppData\Local\Temp\ipykernel_12452\4019313650.py:4: SyntaxWarning: invalid escape sequence '\g'
  genre_list_df = pd.read_csv('data\genres_data.csv')


['acoustic',
 'afrobeat',
 'alt-rock',
 'ambient',
 'black-metal',
 'blues',
 'breakbeat',
 'cantopop',
 'chicago-house',
 'chill',
 'classical',
 'club',
 'comedy',
 'country',
 'dance',
 'dancehall',
 'death-metal',
 'deep-house',
 'detroit-techno',
 'disco',
 'drum-and-bass',
 'dub',
 'dubstep',
 'edm',
 'electro',
 'electronic',
 'emo',
 'folk',
 'forro',
 'french',
 'funk',
 'garage',
 'german',
 'gospel',
 'goth',
 'grindcore',
 'groove',
 'guitar',
 'hard-rock',
 'hardcore',
 'hardstyle',
 'heavy-metal',
 'hip-hop',
 'house',
 'indian',
 'indie',
 'industrial',
 'jazz',
 'k-pop',
 'metal',
 'metalcore',
 'minimal-techno',
 'new-age',
 'opera',
 'party',
 'piano',
 'pop',
 'pop-film',
 'power-pop',
 'progressive-house',
 'psych-rock',
 'punk',
 'punk-rock',
 'rock',
 'rock-n-roll',
 'romance',
 'sad',
 'salsa',
 'samba',
 'sertanejo',
 'show-tunes',
 'singer-songwriter',
 'ska',
 'sleep',
 'songwriter',
 'soul',
 'spanish',
 'swedish',
 'tango',
 'techno',
 'trance',
 'trip-hop']

**Create Item-Feature Matrix of User's Playlist**

The item-feature matrix of the database of 1M+ Spotify songs is now ready to be used for the cosine similarity algorithm. Next, create an item-feature matrix of the user's playlist. 

First, read the URL of the user's playlist and gather the songs along with the associated audio features. 

In [3]:
#connect to spotify API
# Set Spotify API credentials
client_id = 'de12bbd9fca54cdea7698229811c98dd'
client_secret = '0fbe3b5e970d46e8832f7e968da90c42'
redirect_uri = 'http://localhost:3000'

#

# Initialize the Spotipy client with authentication
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id, client_secret, redirect_uri))

track_link = 'https://open.spotify.com/track/2PYDUM2IC01FOTDLQdFxJP?si=50be87899a164a07'
track_id = track_link.split('/')[-1].split('?')[0]
# Get the first 50 songs of the playlist
track = sp.track(track_id)

# Create lists to hold track titles and artist names
titles, artists, uri = [], [], []


# Iterate through the tracks and collect title, artist, and uri
titles.append(track['name'])
artist_names = ', '.join([artist['name'] for artist in track['artists']])
artists.append(artist_names)
uri.append(track['uri'])
    
# Create a DataFrame
data = {'Title': titles, 'Artist': artists, 'uri': uri}
playlist = pd.DataFrame(data)

# create new feature columns and assign null values
new_feat = ['danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
for item in new_feat:
    playlist[item] = 0
    

# fill null values with feature values 
for i in range(len(playlist)):
    track_uri = playlist.iloc[i].uri
    audio_features = sp.audio_features(track_uri)
    json_string = json.dumps(audio_features[0])
    dictionary = json.loads(json_string)
    
    #update feature values
    for feature in new_feat:
        # Explicitly cast the column to float64 before assignment
        playlist[feature] = playlist[feature].astype('float64')
        playlist.loc[i, feature] = dictionary[feature]
    
playlist

,Title,Artist,uri,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3 1 0 7 - 2 (feat. DuongG & Nau),"W/N, Duongg, Nau",spotify:track:2PYDUM2IC01FOTDLQdFxJP,0.738,0.422,-9.76,1.0,0.0365,0.851,0.00111,0.712,0.315,122.071


**Finding Genres of Songs in Playlist**

The Spotify API does not have genres for each song, however, they do provide genres for each artist. So for each song, I will use the artist to find the associated genre to the song. 

**Task Parallelization**

To do this, I would have to iterate through each song and use the Spotify API to find the genre to the artist of the song provided. This process would take a long amount of time. So, I used task parallelization to decrease computation times. By default, when you run a Python script, it typically utilizes a single CPU, operating on a single processing unit. To use the power of available CPUs, I integrated the Python package Joblib for parallel processing. This significantly enhances program efficiency by distributing functions across multiple CPUs, resulting in an average of 30.7% reduction in processing times.

In [4]:
# Create a list of artist names from the playlist
artist_names = playlist['Artist'].tolist()

# Create an empty list to store genres
genres = []

# Parallelization process function to iterate through artist names and retrieve genres
def process_artist(artist_name, sp):
    search_results = sp.search(q=artist_name, type='artist')
    genres_info = []

    if 'artists' in search_results and 'items' in search_results['artists']:
        artists = search_results['artists']['items']

        for artist in artists:
            if artist['name'] == artist_name:
                genres_info = artist.get('genres', [])
                break

    genre_string = ', '.join(genres_info) if genres_info else 'No Genre Found'
    return genre_string

# Fill genre for each song using Parallelization
genres = Parallel(n_jobs=-1)(delayed(process_artist)(artist_name, sp) for artist_name in artist_names)

playlist['Genre'] = genres

playlist

,Title,Artist,uri,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,Genre
0,3 1 0 7 - 2 (feat. DuongG & Nau),"W/N, Duongg, Nau",spotify:track:2PYDUM2IC01FOTDLQdFxJP,0.738,0.422,-9.76,1.0,0.0365,0.851,0.00111,0.712,0.315,122.071,No Genre Found


**Spotify's Unique Genre Names**

The genres that Spotify provides for each artist is very unique. To successfully run the cosine similarity algorithm, I need the genres of both item-feature matrices to match. To do this, I used the list of genres from the item-feature matrix and did a substring search for each of the genres to assign a binary value in the playlist item-feature matrix. 

In [5]:
# using the genre column, find substrings of genres and assign values of 1 if found
# for genre in updated_genre_list:
#     playlist['genre_' + genre] = playlist['genre'].str.contains(genre).astype(int)

# playlist = playlist.drop(columns=['genre'])

# playlist
genre_count = {}

# Substring search of genres, 
for genre in genre_list:
        # substring search for genres, assigns binary value in playlist item-feature matrix
        playlist['genre_'+genre] = playlist['Genre'].str.contains(genre).astype(int)
        # gather count of each genre in playlist
        if playlist['genre_'+genre].sum() > 0:
            genre_count[genre] = playlist['genre_'+genre].sum()
            
playlist = playlist.drop(columns=['Genre'])

#get top 3 genres for recommendation 
top_3_genres = sorted(genre_count, key=genre_count.get, reverse=True)[:3]

top_3_genres
            
playlist

,Title,Artist,uri,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre_acoustic,genre_afrobeat,genre_alt-rock,genre_ambient,genre_black-metal,genre_blues,genre_breakbeat,genre_cantopop,genre_chicago-house,genre_chill,genre_classical,genre_club,genre_comedy,genre_country,genre_dance,genre_dancehall,genre_death-metal,genre_deep-house,genre_detroit-techno,genre_disco,genre_drum-and-bass,genre_dub,genre_dubstep,genre_edm,genre_electro,genre_electronic,genre_emo,genre_folk,genre_forro,genre_french,genre_funk,genre_garage,genre_german,genre_gospel,genre_goth,genre_grindcore,genre_groove,genre_guitar,genre_hard-rock,genre_hardcore,genre_hardstyle,genre_heavy-metal,genre_hip-hop,genre_house,genre_indian,genre_indie,genre_industrial,genre_jazz,genre_k-pop,genre_metal,genre_metalcore,genre_minimal-techno,genre_new-age,genre_opera,genre_party,genre_piano,genre_pop,genre_pop-film,genre_power-pop,genre_progressive-house,genre_psych-rock,genre_punk,genre_punk-rock,genre_rock,genre_rock-n-roll,genre_romance,genre_sad,genre_salsa,genre_samba,genre_sertanejo,genre_show-tunes,genre_singer-songwriter,genre_ska,genre_sleep,genre_songwriter,genre_soul,genre_spanish,genre_swedish,genre_tango,genre_techno,genre_trance,genre_trip-hop
0,3 1 0 7 - 2 (feat. DuongG & Nau),"W/N, Duongg, Nau",spotify:track:2PYDUM2IC01FOTDLQdFxJP,0.738,0.422,-9.76,1.0,0.0365,0.851,0.00111,0.712,0.315,122.071,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
# Need to find the year and popularity of each song in the playlist 
    
playlist['year'] = [0]*len(playlist)
playlist['popularity'] = [0]*len(playlist)

# iterate through each song to find popularity and release year
for index, row in playlist.iterrows():
    track_uri = row['uri']
    # Get audio features of the track
    track_info = sp.track(track_uri)

    # Extract release date from track info
    release_date = track_info['album']['release_date']
    popularity = track_info['popularity']

    # Extract year from release date
    release_year = int(release_date.split('-')[0])

    playlist.loc[index, 'year'] = int(release_year)
    playlist.loc[index,'popularity'] = int(popularity)

playlist

,Title,Artist,uri,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre_acoustic,genre_afrobeat,genre_alt-rock,genre_ambient,genre_black-metal,genre_blues,genre_breakbeat,genre_cantopop,genre_chicago-house,genre_chill,genre_classical,genre_club,genre_comedy,genre_country,genre_dance,genre_dancehall,genre_death-metal,genre_deep-house,genre_detroit-techno,genre_disco,genre_drum-and-bass,genre_dub,genre_dubstep,genre_edm,genre_electro,genre_electronic,genre_emo,genre_folk,genre_forro,genre_french,genre_funk,genre_garage,genre_german,genre_gospel,genre_goth,genre_grindcore,genre_groove,genre_guitar,genre_hard-rock,genre_hardcore,genre_hardstyle,genre_heavy-metal,genre_hip-hop,genre_house,genre_indian,genre_indie,genre_industrial,genre_jazz,genre_k-pop,genre_metal,genre_metalcore,genre_minimal-techno,genre_new-age,genre_opera,genre_party,genre_piano,genre_pop,genre_pop-film,genre_power-pop,genre_progressive-house,genre_psych-rock,genre_punk,genre_punk-rock,genre_rock,genre_rock-n-roll,genre_romance,genre_sad,genre_salsa,genre_samba,genre_sertanejo,genre_show-tunes,genre_singer-songwriter,genre_ska,genre_sleep,genre_songwriter,genre_soul,genre_spanish,genre_swedish,genre_tango,genre_techno,genre_trance,genre_trip-hop,year,popularity
0,3 1 0 7 - 2 (feat. DuongG & Nau),"W/N, Duongg, Nau",spotify:track:2PYDUM2IC01FOTDLQdFxJP,0.738,0.422,-9.76,1.0,0.0365,0.851,0.00111,0.712,0.315,122.071,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021,57


In [7]:
# make buckets based on year to match item-feature matrix of 1M+ song database
 
# Make columns for each time period
playlist['year_2000-2004'] = playlist['year'].apply(lambda year: 1 if year>=2000 and year<2005 else 0)
playlist['year_2005-2009'] = playlist['year'].apply(lambda year: 1 if year>=2005 and year<2010 else 0)
playlist['year_2010-2014'] = playlist['year'].apply(lambda year: 1 if year>=2010 and year<2015 else 0)
playlist['year_2015-2019'] = playlist['year'].apply(lambda year: 1 if year>=2015 and year<2020 else 0)
playlist['year_2020-2023'] = playlist['year'].apply(lambda year: 1 if year>=2020 and year<2025 else 0)
 
# Drop year column, no longer needed
playlist = playlist.drop(columns=['year'])

playlist

,Title,Artist,uri,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre_acoustic,genre_afrobeat,genre_alt-rock,genre_ambient,genre_black-metal,genre_blues,genre_breakbeat,genre_cantopop,genre_chicago-house,genre_chill,genre_classical,genre_club,genre_comedy,genre_country,genre_dance,genre_dancehall,genre_death-metal,genre_deep-house,genre_detroit-techno,genre_disco,genre_drum-and-bass,genre_dub,genre_dubstep,genre_edm,genre_electro,genre_electronic,genre_emo,genre_folk,genre_forro,genre_french,genre_funk,genre_garage,genre_german,genre_gospel,genre_goth,genre_grindcore,genre_groove,genre_guitar,genre_hard-rock,genre_hardcore,genre_hardstyle,genre_heavy-metal,genre_hip-hop,genre_house,genre_indian,genre_indie,genre_industrial,genre_jazz,genre_k-pop,genre_metal,genre_metalcore,genre_minimal-techno,genre_new-age,genre_opera,genre_party,genre_piano,genre_pop,genre_pop-film,genre_power-pop,genre_progressive-house,genre_psych-rock,genre_punk,genre_punk-rock,genre_rock,genre_rock-n-roll,genre_romance,genre_sad,genre_salsa,genre_samba,genre_sertanejo,genre_show-tunes,genre_singer-songwriter,genre_ska,genre_sleep,genre_songwriter,genre_soul,genre_spanish,genre_swedish,genre_tango,genre_techno,genre_trance,genre_trip-hop,popularity,year_2000-2004,year_2005-2009,year_2010-2014,year_2015-2019,year_2020-2023
0,3 1 0 7 - 2 (feat. DuongG & Nau),"W/N, Duongg, Nau",spotify:track:2PYDUM2IC01FOTDLQdFxJP,0.738,0.422,-9.76,1.0,0.0365,0.851,0.00111,0.712,0.315,122.071,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,57,0,0,0,0,1


In [8]:
# apply scaling again this time to playlist dataframe to normalize feature values 
min_row = {'popularity': '0', 'loudness': '-60', 'tempo': '0'}
max_row = {'popularity': '100', 'loudness': '0', 'tempo': '250'}

min_row_df = pd.DataFrame([min_row])
max_row_df = pd.DataFrame([max_row])

playlist = pd.concat([playlist, min_row_df], ignore_index=True)
playlist = pd.concat([playlist, max_row_df], ignore_index=True)

# scale popularity, loudness, and tempo features to 0-1
scale = ['popularity', 'loudness', 'tempo']
scaler = MinMaxScaler()
playlist[scale] = scaler.fit_transform(playlist[scale])

# drop min and max values
playlist = playlist.iloc[:-2]

playlist

,Title,Artist,uri,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre_acoustic,genre_afrobeat,genre_alt-rock,genre_ambient,genre_black-metal,genre_blues,genre_breakbeat,genre_cantopop,genre_chicago-house,genre_chill,genre_classical,genre_club,genre_comedy,genre_country,genre_dance,genre_dancehall,genre_death-metal,genre_deep-house,genre_detroit-techno,genre_disco,genre_drum-and-bass,genre_dub,genre_dubstep,genre_edm,genre_electro,genre_electronic,genre_emo,genre_folk,genre_forro,genre_french,genre_funk,genre_garage,genre_german,genre_gospel,genre_goth,genre_grindcore,genre_groove,genre_guitar,genre_hard-rock,genre_hardcore,genre_hardstyle,genre_heavy-metal,genre_hip-hop,genre_house,genre_indian,genre_indie,genre_industrial,genre_jazz,genre_k-pop,genre_metal,genre_metalcore,genre_minimal-techno,genre_new-age,genre_opera,genre_party,genre_piano,genre_pop,genre_pop-film,genre_power-pop,genre_progressive-house,genre_psych-rock,genre_punk,genre_punk-rock,genre_rock,genre_rock-n-roll,genre_romance,genre_sad,genre_salsa,genre_samba,genre_sertanejo,genre_show-tunes,genre_singer-songwriter,genre_ska,genre_sleep,genre_songwriter,genre_soul,genre_spanish,genre_swedish,genre_tango,genre_techno,genre_trance,genre_trip-hop,popularity,year_2000-2004,year_2005-2009,year_2010-2014,year_2015-2019,year_2020-2023
0,3 1 0 7 - 2 (feat. DuongG & Nau),"W/N, Duongg, Nau",spotify:track:2PYDUM2IC01FOTDLQdFxJP,0.738,0.422,0.837333,1.0,0.0365,0.851,0.00111,0.712,0.315,0.488284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.57,0.0,0.0,0.0,0.0,1.0


In [9]:
# sort the dataframes in alphabetical order so columns correspond to each other for the cosine similarity algorithm
playlist = playlist.sort_index(axis=1)
feat_vec = feat_vec.sort_index(axis=1)

# for cosine similarity, drop track_id column of the dataframe, this is not needed and numerical values are only needed
feat_vec_cosine_sim = feat_vec.drop('track_id', axis=1)

# drop the Artist, Title, and uri in the playlist dataframe as well since they are not numerical values  
columns_dropped = ['Artist', 'Title', 'uri']
playlist_cosine_sim = playlist.drop(columns_dropped, axis=1)


In [10]:
# Calculate column averages of the playlist dataframe
column_averages = playlist_cosine_sim.mean()

# Create a new DataFrame for the averages and totals
averages_cosine_sim = pd.DataFrame([column_averages], index=['Average'])

averages_cosine_sim

,acousticness,danceability,energy,genre_acoustic,genre_afrobeat,genre_alt-rock,genre_ambient,genre_black-metal,genre_blues,genre_breakbeat,genre_cantopop,genre_chicago-house,genre_chill,genre_classical,genre_club,genre_comedy,genre_country,genre_dance,genre_dancehall,genre_death-metal,genre_deep-house,genre_detroit-techno,genre_disco,genre_drum-and-bass,genre_dub,genre_dubstep,genre_edm,genre_electro,genre_electronic,genre_emo,genre_folk,genre_forro,genre_french,genre_funk,genre_garage,genre_german,genre_gospel,genre_goth,genre_grindcore,genre_groove,genre_guitar,genre_hard-rock,genre_hardcore,genre_hardstyle,genre_heavy-metal,genre_hip-hop,genre_house,genre_indian,genre_indie,genre_industrial,genre_jazz,genre_k-pop,genre_metal,genre_metalcore,genre_minimal-techno,genre_new-age,genre_opera,genre_party,genre_piano,genre_pop,genre_pop-film,genre_power-pop,genre_progressive-house,genre_psych-rock,genre_punk,genre_punk-rock,genre_rock,genre_rock-n-roll,genre_romance,genre_sad,genre_salsa,genre_samba,genre_sertanejo,genre_show-tunes,genre_singer-songwriter,genre_ska,genre_sleep,genre_songwriter,genre_soul,genre_spanish,genre_swedish,genre_tango,genre_techno,genre_trance,genre_trip-hop,instrumentalness,liveness,loudness,mode,popularity,speechiness,tempo,valence,year_2000-2004,year_2005-2009,year_2010-2014,year_2015-2019,year_2020-2023
Average,0.851,0.738,0.422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00111,0.712,0.837333,1.0,0.57,0.0365,0.488284,0.315,0.0,0.0,0.0,0.0,1.0


In [11]:
top_3_genres

[]

In [14]:
#generate similarity scores!
similarity_scores = cosine_similarity(feat_vec_cosine_sim, averages_cosine_sim)
 
feat_vec['similarity_score'] = similarity_scores
 
#sort df from highest to lowest by similarity score and to show songs with highest similarity scores
top_similarities = feat_vec.sort_values(by='similarity_score', ascending=False)

#remove rows in recommendations from top_similarities where IDs match with playlist IDs, this makes sure that no recommendation is already in the user's playlist
top_similarities = top_similarities[~top_similarities['track_id'].isin(playlist['uri'])]

# Check if top_3_genres contains at least 3 genres
if len(top_3_genres) >= 3:
    # Get song recs from top 3 genres
    first_genre = top_similarities.loc[top_similarities['genre_'+top_3_genres[0]] == 1].head(45)
    second_genre = top_similarities.loc[top_similarities['genre_'+top_3_genres[1]] == 1].head(30)
    third_genre = top_similarities.loc[top_similarities['genre_'+top_3_genres[2]] == 1].head(15)
    
    top_similarities = pd.concat([first_genre, second_genre, third_genre], ignore_index=True)
else:
    if len(top_3_genres) == 2:
        # If there are only 2 genres, adjust the number of recommendations accordingly
        first_genre = top_similarities.loc[top_similarities['genre_'+top_3_genres[0]] == 1].head(60)
        second_genre = top_similarities.loc[top_similarities['genre_'+top_3_genres[1]] == 1].head(40)
        top_similarities = pd.concat([first_genre, second_genre], ignore_index=True)
    else:
        if len(top_3_genres) == 1:
        # If there is only 1 genre, adjust the number of recommendations accordingly
            first_genre = top_similarities.loc[top_similarities['genre_'+top_3_genres[0]] == 1].head(100)
            top_similarities = first_genre
        elif len(top_3_genres) == 0:
            # If there are no genres, get the top 100 recommendations
            top_similarities = top_similarities.head(100)

top_similarities

,acousticness,danceability,energy,genre_acoustic,genre_afrobeat,genre_alt-rock,genre_ambient,genre_black-metal,genre_blues,genre_breakbeat,genre_cantopop,genre_chicago-house,genre_chill,genre_classical,genre_club,genre_comedy,genre_country,genre_dance,genre_dancehall,genre_death-metal,genre_deep-house,genre_detroit-techno,genre_disco,genre_drum-and-bass,genre_dub,genre_dubstep,genre_edm,genre_electro,genre_electronic,genre_emo,genre_folk,genre_forro,genre_french,genre_funk,genre_garage,genre_german,genre_gospel,genre_goth,genre_grindcore,genre_groove,genre_guitar,genre_hard-rock,genre_hardcore,genre_hardstyle,genre_heavy-metal,genre_hip-hop,genre_house,genre_indian,genre_indie,genre_industrial,genre_jazz,genre_k-pop,genre_metal,genre_metalcore,genre_minimal-techno,genre_new-age,genre_opera,genre_party,genre_piano,genre_pop,genre_pop-film,genre_power-pop,genre_progressive-house,genre_psych-rock,genre_punk,genre_punk-rock,genre_rock,genre_rock-n-roll,genre_romance,genre_sad,genre_salsa,genre_samba,genre_sertanejo,genre_show-tunes,genre_singer-songwriter,genre_ska,genre_sleep,genre_songwriter,genre_soul,genre_spanish,genre_swedish,genre_tango,genre_techno,genre_trance,genre_trip-hop,instrumentalness,liveness,loudness,mode,popularity,speechiness,tempo,track_id,valence,year_2000-2004,year_2005-2009,year_2010-2014,year_2015-2019,year_2020-2024,similarity_score
576607,0.723,0.627,0.532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.934,0.805960,1.0,0.65,0.0413,0.579232,5mUH00xAH26BsmUvPxW2yk,0.433,0.0,0.0,0.0,0.0,1.0,0.912388
574238,0.964,0.774,0.331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000046,0.607,0.762649,1.0,0.67,0.0422,0.491796,7qHPpmIEa0YRnMf1d0XoWj,0.599,0.0,0.0,0.0,0.0,1.0,0.911106
498861,0.702,0.770,0.634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000269,0.645,0.890497,1.0,0.50,0.0382,0.519844,55MzHjHYq3N6V1T97TuKdA,0.499,0.0,0.0,0.0,0.0,1.0,0.909707
538175,0.882,0.830,0.393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002960,0.683,0.707701,1.0,0.39,0.1850,0.540520,2cx1kaxO0mwCRGcrq73U5v,0.421,0.0,0.0,0.0,0.0,1.0,0.908101
469059,0.733,0.648,0.456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.956,0.810585,1.0,0.45,0.0346,0.439068,3BUQz5w70FNxtdfEt2UM5K,0.374,0.0,0.0,0.0,0.0,1.0,0.907856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [15]:
# find the track name, artist, and 30s audio preview or each song using the track_id
top_similarities['track'] = [None]*len(top_similarities)
top_similarities['artist'] = [None]*len(top_similarities)
top_similarities['preview'] = [None]*len(top_similarities)

# get track name, artist, and 30s audio clip url
# Find the track name, artist, and 30s audio preview for each song using the track_id
for index, row in top_similarities.iterrows():
    track_info = sp.track(row['track_id'])
    track_name = track_info['name']
    artist_name = track_info['artists'][0]['name']
    preview_url = track_info['preview_url']
    
    # Update DataFrame with track information
    top_similarities.at[index, 'track'] = track_name
    top_similarities.at[index, 'artist'] = artist_name
    top_similarities.at[index, 'preview'] = preview_url



In [16]:
# Check if top_similarities DataFrame is not empty
if not top_similarities.empty:
    # Drop rows with missing values in track, artist, or preview columns
    top_similarities.dropna(subset=['track', 'artist', 'preview'], inplace=True)
    
    # Iterate over DataFrame rows and print track information
    for index, row in top_similarities.iterrows():
        track_name = row['track']
        artist_name = row['artist']
        preview_url = row['preview']
        print(track_name, "| " + artist_name, "| ", preview_url)
else:
    print("Error: top_similarities DataFrame is empty.")


Chegou um Áudio - Ao Vivo | Faixa Bônus | Israel & Rodolffo |  https://p.scdn.co/mp3-preview/25e68ab662d51127343d72cf0769f36bb0b0e84e?cid=de12bbd9fca54cdea7698229811c98dd
QUE DIABLOS HICE (INTERLUDE) | Eslabon Armado |  https://p.scdn.co/mp3-preview/ace97347c7fe7fdf61511af969b9ed17b23e45ee?cid=de12bbd9fca54cdea7698229811c98dd
Black Lança | Dj C4 |  https://p.scdn.co/mp3-preview/bc124ddce91a173d0add1044ba41dbe2ee51533f?cid=de12bbd9fca54cdea7698229811c98dd
things u did to me | sorrow |  https://p.scdn.co/mp3-preview/4f4de005a36e06e675e5a5e5a75748f78e4b7c9c?cid=de12bbd9fca54cdea7698229811c98dd
Janelinha - Ao Vivo | Zé Neto & Cristiano |  https://p.scdn.co/mp3-preview/74e66ac7d358d2c7800c6efd97db8a9c900b85fa?cid=de12bbd9fca54cdea7698229811c98dd
Garrafa Acompanhada - Acústico | Rafael Frare |  https://p.scdn.co/mp3-preview/afc99aa373ed9d49609e73c4c2aa172da369ce0a?cid=de12bbd9fca54cdea7698229811c98dd
Vou Morar no Cabaré | GORDÃO DO PC |  https://p.scdn.co/mp3-preview/3402a81438f08474930725df

In [17]:
# Get genres of each track in playlist
artist_names = top_similarities['artist'].tolist()
    
# Create an empty list to store genres
genres = []

# Fill genre for each song using Parallelization
genres = Parallel(n_jobs=1)(delayed(process_artist)(artist_name, sp) for artist_name in artist_names)

# Add genres to the dataframe
top_similarities['genre'] = genres

# if songs in recs have any ethnic songs
ethnic_genres = ['colombia', 'latin', 'mexican', 'puerto rican', 'dominican', 'italian', 'spanish', 'brasil', 'argentine', 'anime', 'japanese', 'indonesian', 'vietnamese', 'korean', 'chinese', 'taiwan', 'spanish']
    
# remove any songs that have ethnic genres included
mask = top_similarities['genre'].str.contains('|'.join(ethnic_genres), case=False)
top_similarities.drop(top_similarities[mask].index, inplace=True)

# Check if top_3_genres contains at least 3 genres
if len(top_3_genres) >= 3:
    # Get song recs from top 3 genres
    first_genre = top_similarities.loc[top_similarities['genre_'+top_3_genres[0]] == 1].head(5)
    second_genre = top_similarities.loc[top_similarities['genre_'+top_3_genres[1]] == 1].head(3)
    third_genre = top_similarities.loc[top_similarities['genre_'+top_3_genres[2]] == 1].head(2)

    top_similarities = pd.concat([first_genre, second_genre, third_genre], ignore_index=True)
else:
    if len(top_3_genres) == 2:
        # If there are only 2 genres, adjust the number of recommendations accordingly
        first_genre = top_similarities.loc[top_similarities['genre_'+top_3_genres[0]] == 1].head(6)
        second_genre = top_similarities.loc[top_similarities['genre_'+top_3_genres[1]] == 1].head(4)

        top_similarities = pd.concat([first_genre, second_genre], ignore_index=True)
    elif len(top_3_genres) == 1:
        # If there is only 1 genre, adjust the number of recommendations accordingly
        first_genre = top_similarities.loc[top_similarities['genre_'+top_3_genres[0]] == 1].head(10)

        top_similarities = first_genre
    
top_similarities

,acousticness,danceability,energy,genre_acoustic,genre_afrobeat,genre_alt-rock,genre_ambient,genre_black-metal,genre_blues,genre_breakbeat,genre_cantopop,genre_chicago-house,genre_chill,genre_classical,genre_club,genre_comedy,genre_country,genre_dance,genre_dancehall,genre_death-metal,genre_deep-house,genre_detroit-techno,genre_disco,genre_drum-and-bass,genre_dub,genre_dubstep,genre_edm,genre_electro,genre_electronic,genre_emo,genre_folk,genre_forro,genre_french,genre_funk,genre_garage,genre_german,genre_gospel,genre_goth,genre_grindcore,genre_groove,genre_guitar,genre_hard-rock,genre_hardcore,genre_hardstyle,genre_heavy-metal,genre_hip-hop,genre_house,genre_indian,genre_indie,genre_industrial,genre_jazz,genre_k-pop,genre_metal,genre_metalcore,genre_minimal-techno,genre_new-age,genre_opera,genre_party,genre_piano,genre_pop,genre_pop-film,genre_power-pop,genre_progressive-house,genre_psych-rock,genre_punk,genre_punk-rock,genre_rock,genre_rock-n-roll,genre_romance,genre_sad,genre_salsa,genre_samba,genre_sertanejo,genre_show-tunes,genre_singer-songwriter,genre_ska,genre_sleep,genre_songwriter,genre_soul,genre_spanish,genre_swedish,genre_tango,genre_techno,genre_trance,genre_trip-hop,instrumentalness,liveness,loudness,mode,popularity,speechiness,tempo,track_id,valence,year_2000-2004,year_2005-2009,year_2010-2014,year_2015-2019,year_2020-2024,similarity_score,track,artist,preview,genre
576607,0.723,0.627,0.532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.934,0.805960,1.0,0.65,0.0413,0.579232,5mUH00xAH26BsmUvPxW2yk,0.433,0.0,0.0,0.0,0.0,1.0,0.912388,Chegou um Áudio - Ao Vivo | Faixa Bônus,Israel & Rodolffo,https://p.scdn.co/mp3-preview/25e68ab662d51127...,"agronejo, arrocha, sertanejo, sertanejo pop, s..."
574238,0.964,0.774,0.331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000046,0.607,0.762649,1.0,0.67,0.0422,0.491796,7qHPpmIEa0YRnMf1d0XoWj,0.599,0.0,0.0,0.0,0.0,1.0,0.911106,QUE DIABLOS HICE (INTERLUDE),Eslabon Armado,https://p.scdn.co/mp3-preview/ace97347c7fe7fdf...,"corrido, corridos tumbados, sad sierreno, sier..."
498861,0.702,0.770,0.634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000269,0.645,0.890497,1.0,0.50,0.0382,0.519844,55MzHjHYq3N6V1T97TuKdA,0.499,0.0,0.0,0.0,0.0,1.0,0.909707,Black Lança,Dj C4,https://p.scdn.co/mp3-preview/bc124ddce91a173d...,"funk mandelao, funk viral"
538175,0.882,0.830,0.393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002960,0.683,0.707701,1.0,0.39,0.1850,0.540520,2cx1kaxO0mwCRGcrq73U5v,0.421,0.0,0.0,0.0,0.0,1.0,0.908101,things u did to me,sorrow,https://p.scdn.co/mp3-preview/4f4de005a36e06e6...,"lo-fi chill, sad lo-fi, sad rap"
469059,0.733,0.648,0.456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [18]:
#show only specific columns useful to the user 
display_features = ['track', 'artist', 'similarity_score', 'genre', 'preview']

playlist_recs = top_similarities[display_features]

playlist_recs['similarity_score'] = (playlist_recs['similarity_score']*100).round(2)

playlist_recs

C:\Users\Quan\AppData\Local\Temp\ipykernel_12452\1613625715.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlist_recs['similarity_score'] = (playlist_recs['similarity_score']*100).round(2)


,track,artist,similarity_score,genre,preview
576607,Chegou um Áudio - Ao Vivo | Faixa Bônus,Israel & Rodolffo,91.24,"agronejo, arrocha, sertanejo, sertanejo pop, s...",https://p.scdn.co/mp3-preview/25e68ab662d51127...
574238,QUE DIABLOS HICE (INTERLUDE),Eslabon Armado,91.11,"corrido, corridos tumbados, sad sierreno, sier...",https://p.scdn.co/mp3-preview/ace97347c7fe7fdf...
498861,Black Lança,Dj C4,90.97,"funk mandelao, funk viral",https://p.scdn.co/mp3-preview/bc124ddce91a173d...
538175,things u did to me,sorrow,90.81,"lo-fi chill, sad lo-fi, sad rap",https://p.scdn.co/mp3-preview/4f4de005a36e06e6...
469059,Janelinha - Ao Vivo,Zé Neto & Cristiano,90.79,"agronejo, arrocha, sertanejo, sertanejo univer...",https://p.scdn.co/mp3-preview/74e66ac7d358d2c7...
...,...,...,...,...,...
497635,Just the Two of Us,Sunmoon,89.79,"chill house, tropical house",https://p.scdn.co/mp3-preview/79a85f651b311767...
523077,Mistérios / Tempo ao Tempo (Ao Vivo),Ícaro e Gilmar,89.79,"agronejo, sertanejo pop",https://p.scdn.co/mp3-preview/9e37667e3a307431...
523010,Na Hora H / Pra Não Pensar em Você (Ao Vivo),Ícaro e Gilmar,89.79,"agronejo, sertanejo pop",https://p.scdn.co/mp3-preview/6fe7923afb9a9da3...
562164,broken cd,beabadoobee,89.79,"bedroom pop, bubblegrunge, indie pop, pov: indie",https://p.scdn.co/mp3-preview/f9c17125fcb28ae0...
